In [ ]:
# imports
from google.colab import drive
import time

In [ ]:
drive.mount('/content/drive')
DATA_PATH = "/content/drive/My Drive"
infile = open(DATA_PATH+'/combined_cleaned.pkl','rb')
#infile = open(DATA_PATH+'/combined_df.pkl','rb')
df = pickle.load(infile)

In [ ]:
df

In [ ]:
#Gather and Prepare the Data
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

texts = df['Email Text'].values
labels = df['Email Type'].values

# Split the data into training and test sets
texts_train, texts_test, labels_train, labels_test = train_test_split(texts, labels, test_size=0.2, random_state=42)

# Preprocess the text data: tokenization and padding
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(texts_train)

sequences_train = tokenizer.texts_to_sequences(texts_train)
sequences_test = tokenizer.texts_to_sequences(texts_test)

# Pad sequences to ensure uniform input size
X_train = pad_sequences(sequences_train, maxlen=200)
X_test = pad_sequences(sequences_test, maxlen=200)


In [ ]:
# set model architecture
# LSTM

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=50, input_length=200))
model.add(LSTM(64))
model.add(Dense(1, activation='sigmoid'))

model.summary()

In [ ]:
#Set Up TensorFlow and Preprocess the Data
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
#train
history = model.fit(X_train, labels_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)


In [ ]:
# Evaluate
test_loss, test_acc = model.evaluate(X_test, labels_test)

print(f"Test Accuracy: {test_acc}")

# plot the training history to visualize the training and validation accuracy/loss over epochs
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')

plt.show()
